# We need to map the objects of the images into existing ontology, steps for mapping:


In [81]:
#pip install owlready2
# pip install pyyaml

Load an ontology from a local repository, or from Internet:

In [15]:
#pip install -U sentence-transformers

# 1.finding the suitable ontology for our objects (here, we select the mason ontology)

In [1]:
#here the file at "E://Jacobs 3rd sem//advproj2//main folder//mason.owl" is the modified file which contains manually inserted classes and sub classes as per requirement -
#like robot, printer3d and their body parts etc. Download MASON ontology file from net-saref4inma ontology 
from owlready2 import *
from sentence_transformers import SentenceTransformer, util
from yaml.loader import SafeLoader
import yaml
from neo4j import GraphDatabase
import json
import ast
# from datetime import datetime
import torch
import glob

onto_path.append("")
onto = get_ontology("E://Jacobs 3rd sem//advproj2//main folder//mason.owl")
onto.load()

get_ontology("http://www.owl-ontologies.com/mason.owl#")

# 2.extract the list of ontology classes using owlready2

In [2]:
def get_ontoCls():
    """
    This function returns the list of classes in the ontology
    Parameter: none
    Return: List
    """
    return list(onto.classes())

In [3]:
get_ontoCls()

[mason.Manufacturing_operation,
 mason.Geometric_entity_for_manufacturing,
 mason.Raw_material,
 mason.System,
 mason.Finished_part,
 mason.Set,
 mason.Semi-finished_part,
 mason.Geographic_resource,
 mason.Material_resource,
 mason.Machine_resource,
 mason.Logistic_operation,
 mason.Human_resource,
 mason.Operation,
 mason.Blank,
 mason.Entity_geometric_flaw,
 mason.Shape,
 mason.Assembly_entity,
 mason.Symmetry,
 mason.Cost_entity,
 mason.Tool,
 mason.Machine-tool,
 mason.Geometric_flaw_between_entities,
 mason.Topology,
 mason.Administrative_entity,
 mason.Technological_entity,
 mason.Geometric_flaw_2nd_order,
 mason.Resource,
 mason.Geometric_flaw_3rd_order,
 mason.Geometric_flaw_4th_order,
 mason.Part,
 mason.Manufacturing_concept,
 mason.Geometric_flaw,
 mason.Geometric_entity,
 mason.Entity,
 mason.A-Sx,
 mason.Casted_aluminium,
 mason.Aluminium_alloy,
 mason.A-Uy,
 mason.Abrasion,
 mason.Shearing_Operation,
 mason.Anodic_dissolving,
 mason.Cutting,
 mason.Evaporative_or_Project

# 3. Extract the objects from images dynamically.


In [ ]:
# first-extract json files data

In [4]:

# Place your JSON data in a directory named 'data/'
def getImg_objs(src):
    """
    This function extracts the objects( machine parts) from the JSON file(Input file) and return the desired filtered objects
    Parameter: src (Link of local directory)
    Parameter: none
    Return: List
    """
    src = src
#     date = datetime.now()
    data = []

    # Change the glob if you want to only look through files with specific names
    files = glob.glob(f'{src}*', recursive=True)
    print('file is:',files)

    for single_file in files:

    #     print(single_file)
        with open(single_file, 'r') as f:

            # Use 'try-except' to skip files that may be missing data
            try:
                json_file = json.load(f)
                data.append([
                  json_file['img_id'],
                  json_file['img_name'],          
                  json_file['detection']
                ])
            except KeyError:
                print(f'Skipping {single_file}')

    # Sort the data
    data.sort()
    return data

In [5]:
#Create a folders in jupyter namely 'data_machine_parts' and insert JSON files containing image info with machine parts.
data_machine_parts = getImg_objs("data_machine_parts/")
data_machine_parts

file is: ['data_machine_parts\\IMG_0235.json', 'data_machine_parts\\IMG_0444.json', 'data_machine_parts\\IMG_0902.json', 'data_machine_parts\\IMG_1045.json']


[['18d132a4d5f1b933009a2beccc30b6003c060e585ba09a329b91e8ee0fa1440a',
  'IMG_1045',
  [{'det_cat': 18,
    'det_name': 'MillingMachine_Controls',
    'det_x': 0.1509166657924652,
    'det_y': 0.25075000524520874,
    'det_w': 0.07383333146572113,
    'det_h': 0.20000000298023224,
    'det_score': 0.3691582977771759},
   {'det_cat': 21,
    'det_name': 'MillingMachine_WorkTable',
    'det_x': 0.17775000631809235,
    'det_y': 0.49024999141693115,
    'det_w': 0.21783334016799927,
    'det_h': 0.10949999839067459,
    'det_score': 0.3937746286392212}]],
 ['229fc139ce71d47f4a005e7016cdd35b7d412855ab994ada2cedd5b25128dfb4',
  'IMG_0444',
  [{'det_cat': 23,
    'det_name': 'Robot_Head',
    'det_x': 0.6015833616256714,
    'det_y': 0.7360000014305115,
    'det_w': 0.04283333197236061,
    'det_h': 0.13199999928474426,
    'det_score': 0.25045061111450195},
   {'det_cat': 1,
    'det_name': 'Printer3D_Filament',
    'det_x': 0.6504999995231628,
    'det_y': 0.5575000047683716,
    'det_w': 0

In [6]:
#Create a folders in jupyter namely 'data_machine' and insert JSON files containing image info with machines.
data_machine = getImg_objs("data_machine/")
data_machine

file is: ['data_machine\\IMG_0444.json', 'data_machine\\IMG_0902.json', 'data_machine\\IMG_1045.json', 'data_machine\\IMG_1070.json']


[['18d132a4d5f1b933009a2beccc30b6003c060e585ba09a329b91e8ee0fa1440a',
  'IMG_1045',
  [{'det_cat': 0,
    'det_name': 'BandSaw',
    'det_x': 0.6184999942779541,
    'det_y': 0.4339999854564667,
    'det_w': 0.1850000023841858,
    'det_h': 0.3490000069141388,
    'det_score': 0.25298449397087097},
   {'det_cat': 4,
    'det_name': 'MillingMachine',
    'det_x': 0.8725833296775818,
    'det_y': 0.4338749945163727,
    'det_w': 0.2548333406448364,
    'det_h': 0.8612499833106995,
    'det_score': 0.375745952129364},
   {'det_cat': 4,
    'det_name': 'MillingMachine',
    'det_x': 0.44200000166893005,
    'det_y': 0.37162500619888306,
    'det_w': 0.20233333110809326,
    'det_h': 0.4637500047683716,
    'det_score': 0.39933040738105774},
   {'det_cat': 4,
    'det_name': 'MillingMachine',
    'det_x': 0.29116666316986084,
    'det_y': 0.4026249945163727,
    'det_w': 0.23733332753181458,
    'det_h': 0.46974998712539673,
    'det_score': 0.40608200430870056},
   {'det_cat': 4,
    'det_

In [50]:
# data[0]

# 4. Creating object properties

In [7]:
def open_saveYaml(path_yaml):
    """
    This function extracts the objects( machine and its parts) from the YAML file(Input file) and return the desired filtered objects.
    Parameter: path_yaml (str: path of the file)
    Return: obj_list(List)
    """
    with open(path_yaml) as f:
        data=yaml.load(f, Loader=SafeLoader)
        #to get the list of names to extract classes/instances from it
        obj_list=data.get('names')
        return obj_list
        
path_="E://Jacobs 3rd sem//advproj2//main folder//data.yaml"
yamldata=open_saveYaml(path_)
yamldata=['onto.'+i for i in yamldata]
yamldata

['onto.Printer3D_Body',
 'onto.Printer3D_Filament',
 'onto.BandSaw_Controls',
 'onto.BandSaw_Saw',
 'onto.BandSaw_WorkTable',
 'onto.Conveyors_BaseRoller',
 'onto.Conveyors_Bed',
 'onto.Conveyors_SlantRoller',
 'onto.Drillpress_Controls',
 'onto.Drillpress_Drill',
 'onto.Drillpress_HandWheel',
 'onto.Drillpress_MachineStand',
 'onto.Drillpress_WorkTable',
 'onto.Lathe_Bed',
 'onto.Lathe_ChipPan',
 'onto.Lathe_Controls',
 'onto.Lathe_CrossSlide',
 'onto.MillingMachine_ChipCollectorPlate',
 'onto.MillingMachine_Controls',
 'onto.MillingMachine_HandWheel',
 'onto.MillingMachine_MachineStand',
 'onto.MillingMachine_WorkTable',
 'onto.Robot_BaseArm',
 'onto.Robot_Head',
 'onto.Robot_UpperArm']

In [8]:
#to get only classes, split each names and get the first word before '_'. Used 'set' to get the distinct vales of classes
def get_clsYaml(yamldata):
    """
    This function extracts the objects( machines) from the YAML file(Input file) and return the desired filtered objects.
    Parameter: yamldata (List: list of machine parts)
    Return: class_list(List)
    """
    cls_lst = set([x.split('_')[0] for x in yamldata])
    class_list=[]                                                      #label list
    #to make a list of classes.
    for s in cls_lst: class_list.append(s) 
    return class_list
    
class_list=get_clsYaml(yamldata)
# class_list=['onto.'+i for i in class_list]
class_list

['onto.Lathe',
 'onto.Drillpress',
 'onto.BandSaw',
 'onto.MillingMachine',
 'onto.Printer3D',
 'onto.Robot',
 'onto.Conveyors']

In [ ]:
# creating object property

In [11]:

for i in class_list:
    for j in yamldata:
        if i == j.split('_')[0]:
            print(i ,' ', j)
            with onto:    
                class has(onto[i.split(".")[1]] >> onto[j.split(".")[1]]):        
                    pass

onto.Lathe   onto.Lathe_Bed
onto.Lathe   onto.Lathe_ChipPan
onto.Lathe   onto.Lathe_Controls
onto.Lathe   onto.Lathe_CrossSlide
onto.Drillpress   onto.Drillpress_Controls
onto.Drillpress   onto.Drillpress_Drill
onto.Drillpress   onto.Drillpress_HandWheel
onto.Drillpress   onto.Drillpress_MachineStand
onto.Drillpress   onto.Drillpress_WorkTable
onto.BandSaw   onto.BandSaw_Controls
onto.BandSaw   onto.BandSaw_Saw
onto.BandSaw   onto.BandSaw_WorkTable
onto.MillingMachine   onto.MillingMachine_ChipCollectorPlate
onto.MillingMachine   onto.MillingMachine_Controls
onto.MillingMachine   onto.MillingMachine_HandWheel
onto.MillingMachine   onto.MillingMachine_MachineStand
onto.MillingMachine   onto.MillingMachine_WorkTable
onto.Printer3D   onto.Printer3D_Body
onto.Printer3D   onto.Printer3D_Filament
onto.Robot   onto.Robot_BaseArm
onto.Robot   onto.Robot_Head
onto.Robot   onto.Robot_UpperArm
onto.Conveyors   onto.Conveyors_BaseRoller
onto.Conveyors   onto.Conveyors_Bed
onto.Conveyors   onto.Con

In [12]:
has.domain

[mason.Lathe,
 mason.Drillpress,
 mason.BandSaw,
 mason.MillingMachine,
 mason.Printer3D,
 mason.Robot,
 mason.Conveyors,
 mason.Conveyors]

In [13]:
has.range

[mason.Lathe_Bed,
 mason.Lathe_ChipPan,
 mason.Lathe_Controls,
 mason.Lathe_CrossSlide,
 mason.Drillpress_Controls,
 mason.Drillpress_Drill,
 mason.Drillpress_HandWheel,
 mason.Drillpress_MachineStand,
 mason.Drillpress_WorkTable,
 mason.BandSaw_Controls,
 mason.BandSaw_Saw,
 mason.BandSaw_WorkTable,
 mason.MillingMachine_ChipCollectorPlate,
 mason.MillingMachine_Controls,
 mason.MillingMachine_HandWheel,
 mason.MillingMachine_MachineStand,
 mason.MillingMachine_WorkTable,
 mason.Printer3D_Body,
 mason.Printer3D_Filament,
 mason.Robot_BaseArm,
 mason.Robot_Head,
 mason.Robot_UpperArm,
 mason.Conveyors_BaseRoller,
 mason.Conveyors_Bed,
 mason.Conveyors_SlantRoller,
 mason.Conveyors_SlantRoller]

# 5. creating and populating instances of respective classes dynamically.

In [14]:
def create_pop(data):
    """
    This function creates instances of machine and its parts that are obtained from the object list containing machine, its parts and configuration. 
    Parameter: data (List: list of machine parts and its configuration)
    Return: none
    """
    for i in range(0,len(data)):
        d_len=len(data[i][2])
        for j in range(0,d_len):
            img_obj = data[i][2][j].get('det_name')
            inst  = onto[img_obj](str(data[i][2][j]))
            print(img_obj,' ', str(data[i][2][j]))

In [15]:
create_pop(data_machine_parts) 

MillingMachine_Controls   {'det_cat': 18, 'det_name': 'MillingMachine_Controls', 'det_x': 0.1509166657924652, 'det_y': 0.25075000524520874, 'det_w': 0.07383333146572113, 'det_h': 0.20000000298023224, 'det_score': 0.3691582977771759}
MillingMachine_WorkTable   {'det_cat': 21, 'det_name': 'MillingMachine_WorkTable', 'det_x': 0.17775000631809235, 'det_y': 0.49024999141693115, 'det_w': 0.21783334016799927, 'det_h': 0.10949999839067459, 'det_score': 0.3937746286392212}
Robot_Head   {'det_cat': 23, 'det_name': 'Robot_Head', 'det_x': 0.6015833616256714, 'det_y': 0.7360000014305115, 'det_w': 0.04283333197236061, 'det_h': 0.13199999928474426, 'det_score': 0.25045061111450195}
Printer3D_Filament   {'det_cat': 1, 'det_name': 'Printer3D_Filament', 'det_x': 0.6504999995231628, 'det_y': 0.5575000047683716, 'det_w': 0.01600000075995922, 'det_h': 0.019999999552965164, 'det_score': 0.25580695271492004}
Robot_Head   {'det_cat': 23, 'det_name': 'Robot_Head', 'det_x': 0.38341665267944336, 'det_y': 0.46224

In [16]:
create_pop(data_machine)

BandSaw   {'det_cat': 0, 'det_name': 'BandSaw', 'det_x': 0.6184999942779541, 'det_y': 0.4339999854564667, 'det_w': 0.1850000023841858, 'det_h': 0.3490000069141388, 'det_score': 0.25298449397087097}
MillingMachine   {'det_cat': 4, 'det_name': 'MillingMachine', 'det_x': 0.8725833296775818, 'det_y': 0.4338749945163727, 'det_w': 0.2548333406448364, 'det_h': 0.8612499833106995, 'det_score': 0.375745952129364}
MillingMachine   {'det_cat': 4, 'det_name': 'MillingMachine', 'det_x': 0.44200000166893005, 'det_y': 0.37162500619888306, 'det_w': 0.20233333110809326, 'det_h': 0.4637500047683716, 'det_score': 0.39933040738105774}
MillingMachine   {'det_cat': 4, 'det_name': 'MillingMachine', 'det_x': 0.29116666316986084, 'det_y': 0.4026249945163727, 'det_w': 0.23733332753181458, 'det_h': 0.46974998712539673, 'det_score': 0.40608200430870056}
MillingMachine   {'det_cat': 4, 'det_name': 'MillingMachine', 'det_x': 0.17216666042804718, 'det_y': 0.46837499737739563, 'det_w': 0.34433332085609436, 'det_h': 0

In [17]:
 # To check for single class:
for i in onto.MillingMachine.instances(): 
    print(i)

mason.{'det_cat': 4, 'det_name': 'MillingMachine', 'det_x': 0.8725833296775818, 'det_y': 0.4338749945163727, 'det_w': 0.2548333406448364, 'det_h': 0.8612499833106995, 'det_score': 0.375745952129364}
mason.{'det_cat': 4, 'det_name': 'MillingMachine', 'det_x': 0.44200000166893005, 'det_y': 0.37162500619888306, 'det_w': 0.20233333110809326, 'det_h': 0.4637500047683716, 'det_score': 0.39933040738105774}
mason.{'det_cat': 4, 'det_name': 'MillingMachine', 'det_x': 0.29116666316986084, 'det_y': 0.4026249945163727, 'det_w': 0.23733332753181458, 'det_h': 0.46974998712539673, 'det_score': 0.40608200430870056}
mason.{'det_cat': 4, 'det_name': 'MillingMachine', 'det_x': 0.17216666042804718, 'det_y': 0.46837499737739563, 'det_w': 0.34433332085609436, 'det_h': 0.8187500238418579, 'det_score': 0.4348146915435791}
mason.{'det_cat': 4, 'det_name': 'MillingMachine', 'det_x': 0.0573333315551281, 'det_y': 0.43575000762939453, 'det_w': 0.1146666631102562, 'det_h': 0.33250001072883606, 'det_score': 0.293686

In [18]:
 # To check for single class:
for i in onto.Robot_Head.instances(): 
    print(i)

mason.{'det_cat': 23, 'det_name': 'Robot_Head', 'det_x': 0.6015833616256714, 'det_y': 0.7360000014305115, 'det_w': 0.04283333197236061, 'det_h': 0.13199999928474426, 'det_score': 0.25045061111450195}
mason.{'det_cat': 23, 'det_name': 'Robot_Head', 'det_x': 0.38341665267944336, 'det_y': 0.4622499942779541, 'det_w': 0.0494999997317791, 'det_h': 0.02449999935925007, 'det_score': 0.27176982164382935}
mason.{'det_cat': 23, 'det_name': 'Robot_Head', 'det_x': 0.3876666724681854, 'det_y': 0.4625000059604645, 'det_w': 0.03266666829586029, 'det_h': 0.05649999901652336, 'det_score': 0.2925967276096344}
mason.{'det_cat': 23, 'det_name': 'Robot_Head', 'det_x': 0.5005833506584167, 'det_y': 0.7021250128746033, 'det_w': 0.03316666558384895, 'det_h': 0.01924999989569187, 'det_score': 0.35125434398651123}
mason.{'det_cat': 23, 'det_name': 'Robot_Head', 'det_x': 0.3851666748523712, 'det_y': 0.4606249928474426, 'det_w': 0.044333335012197495, 'det_h': 0.04125000163912773, 'det_score': 0.42693111300468445}


# 6. Saving ontology

In [21]:
##save ontology- in my case, it is saved at C:\Users\ABHISHEK MATHUR. RDF file is also created at same place that will be used for NEO4J.
def save_onto():
    """
    This function is used to save the ontology
    Parameter: none
    Return: none
    """
    onto.save()
    try:
        onto.save(file = "./ontology_mason.rdf", format = "rdfxml")
    except ValueError:
        print(ValueError)
        
save_onto()
# onto.save(file = "test_mason", format = "rdfxml")

# 7. import the final ontology into Neo4j

In [39]:
# !pip install neo4j 

In [25]:
#Upload ontology to Neo4j for graphing -
#Before this, install neo4j, create a project and name the db as 'neo4j' and password as 'password'.

class Neo4j_connection: #neo4jconnection #used to be COnnect2Neo4j

    def __init__(self, uri, user, pwd, database_name="neo4j"): #connect to the server; create constraint if not exist; default database is "Neo4j"
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        self.__database_name = database_name

        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create driver:", e)

        # check if the neccessary config already existed. if not, create.
        query_string = '''
        CALL db.constraints()
                 '''
        a = self.query(query_string, db= self.__database_name)
        constr = str(a)
        try:
            b = constr[constr.find("description='")+13:constr.find(' ON')]
        except Exception as e:
            pass
        if b == 'CONSTRAINT':
            print("Constraint already existed")
        else:
            try:
                query_string = '''
                    CREATE CONSTRAINT n10s_unique_uri FOR (r:Resource) REQUIRE r.uri IS UNIQUE  
                            '''
                self.query(query_string, db = self.__database_name)
            except Exception as e:
                print("Failed to create constraint - check again", e)

        # check if the neccessary config already existed. if not, create.
        query_string = '''
        match(n) return count(n)
        '''
        a = self.query(query_string, db= self.__database_name)
        strin = str(a[0])
        config_check = int(strin[strin.find("=")+1:strin.find(">")])
        if config_check != 0:
            print("Config already existed")
        else:
            try: #setting up neccessary config for neosemantics
                query_string = '''
                    CALL n10s.graphconfig.init()
                                '''
                self.query(query_string, db= self.__database_name)

            except Exception as e:
                print("Failed to initiate graphconfig: ", e)


    def uploading_orx(self, address, db="neo4j"): #uploading OWL, RDF, and XML #i am trying to
        query_string = (
                "call n10s.rdf.import.fetch('"+address+"','RDF/XML',"+"{"+"verifyUriSyntax: false"+"}"+")")
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            response = list(session.run(query_string))
            print("Sucessfully uploaded XML/RDF/OWL file ")
        except Exception as e:
            print("Upload failed:", e)
        finally:
            if session is not None:
                session.close()
        return response


    def uploading_ttl(self, address, db=None): #uploading Turtle files
        query_string = (
                "call n10s.rdf.import.fetch(" + f'{address}'+  ',"Turtle",{verifyUriSyntax: false})')
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            response = list(session.run(query_string))
            print("Sucessfully uploaded TTL file ")
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response


    def close(self):
        if self.__driver is not None:
            self.__driver.close()

    def query(self, query, db=None): #query commands of choice.
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response

a = Neo4j_connection("bolt://localhost:7687", "neo4j", "password") #("bolt://localhost:7687", "neo4j", "password")
a.uploading_orx("file:///Users/ABHISHEK MATHUR/ontology_mason.rdf", "neo4j")


Query failed: {code: Neo.ClientError.Procedure.ProcedureNotFound} {message: There is no procedure with the name `db.constraints` registered for this database instance. Please ensure you've spelled the procedure name correctly and that the procedure is properly deployed.}
Sucessfully uploaded XML/RDF/OWL file 


[<Record terminationStatus='OK' triplesLoaded=2044 triplesParsed=2044 namespaces={'owl': 'http://www.w3.org/2002/07/owl#', 'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#', 'rdfs': 'http://www.w3.org/2000/01/rdf-schema#', 'ns0': 'http://www.owl-ontologies.com/mason.owl#', 'ns1': 'http://www.w3.org/2003/11/swrlb#'} extraInfo='' callParams={'verifyUriSyntax': False}>]

# Important functions of owlready2

In [32]:
#Access the classes of the ontology, and create new instances / individuals:
# onto.Thing
# list(onto.object_properties())
# list(onto.data_properties())
# list(onto.annotation_properties())
# list(onto.properties())
# list(onto.disjoint_classes() )
# list(onto.disjoint_properties())
# list(onto .disjoints() )
# list(onto.different_individuals())
# print(onto.Lathe.is_a)  #to get parent class
# destroy_entity(onto.Images)
# Owlready classes and individuals can be used as values within search(), as follows:
# onto.search_one(is_a=onto.Images)
#to check the flow of parent and subclasses # .descendants()#.ancestors()
# Robots.descendants() 